In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drivePath = 'drive/MyDrive/Colab Notebooks/finalProj/'

Mounted at /content/drive


In [ ]:
#kaggle API key, to get UD dataset.
! echo "{\"username\":\"ericclark\",\"key\":\"d84149862033d3369bad2f997df47d26\"}" > kaggle.json

!pip install requests
! pip install kaggle
! pip install transformers
! pip install tensorflow

#getting the urban dictionary words datat set from Kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download therohk/urban-dictionary-words-dataset
! unzip urban-dictionary-words-dataset
# gives urbandict-word-defs.csv - for use in defining slang terms

In [ ]:
!head urbandict-word-defs.csv
import csv
file = open("urbandict-word-defs.csv")
csvreader = csv.reader(file)
header = next(csvreader)
print(header)
udWords = []
for row in csvreader:
    udWords.append(row)
print(len(udWords))
file.close()

In [ ]:

# build the slang example sentence dataset.
# random.randint(0, 9)
import random
import requests
import time
import numpy as np
import pickle

datasetSize = 13000
sampleWords = random.sample(udWords, datasetSize)
#['word_id', 'word', 'up_votes', 'down_votes', 'author', 'definition']
response = 0
examples = []
for sampleWord in sampleWords:
    time.sleep(.1)
    response = requests.get("https://api.urbandictionary.com/v0/define?defid="+sampleWord[0])
    apiData = response.json()['list']
    # skip words with no definition
    if(len(apiData) == 0):
      continue
    apiData = apiData[0]
    word = sampleWord[1]
    #print(apiData)
    example = apiData['example']
    examples.append([word, example])
print(len(examples))
  # call the urban dictionary api
  #sleep for .25 seconds
  # add the sentence to the doc.
  # add the tags to the tagset

# convert to dictionary, split sample data on double newline (how the example sentences are split in the api response.)
exampleDict = {}

for row in examples:
  splixEx = row[1].split("\n\n")
  exampleDict[row[0]] = splixEx
#print(exampleDict)
# create a binary pickle file 
f = open("drive/MyDrive/Colab Notebooks/finalProj/examples.pkl","wb")

# write the python object (dict) to pickle file, to save it.
pickle.dump(exampleDict,f)

In [ ]:
# If we end up tokenizing the data, do it here.
# input format is:
# dictionary {
# 'vocab word' : ['example sentence 1', 'example sentence 2', etc.]
# ...   
# }
#
# ouytput format is:
# dictionary {
# 'vocab word' : {'keyToken': ['tokenized', 'key'], 'examples': ['example sentence 1', 'example sentence 2', etc.]}
# ...   
# }
#
def tokenize(dataDict):
  tokenDict = {}
  # for now our simple tokenization will be to split on whitespace.
  for key, exampleSents in exampleDict.items():
    for index, sentence in enumerate(exampleSents):
      exampleSents[index] = sentence.split()
    tokenDict[key.lower()] = {'keyToken' : key.lower().split(), 'examples' : exampleSents}
  return tokenDict

# TODO: when labeling the example sentences, we really should be checking for every single slang word in our vocab, rather than just checking for the example.
# This may cause issues with detecting slang properly, or scoring our data.
def label(dataDict):
  labeledDict = {}
  for key, keyData in dataDict.items():
    exampleLabels = []
    for index, sentence in enumerate(keyData['examples']):
      labels = ['O'] * len(sentence)
      keyLoc = find_subarray(sentence, keyData['keyToken'])
      if keyLoc is not None:
        for idx, location in enumerate(keyLoc):
          # the word is in the sentence.
          # tag that section as slang.
          keyLen = len(keyData['keyToken'])
          for i in range(keyLen):
            slangLabel = 'I-SLANG'
            if i == 0:
              slangLabel = 'B-SLANG'
            labels[location + i] = slangLabel
        exampleLabels.append(labels)
    labeledDict[key] = {'keyToken': keyData['keyToken'], 'examples': keyData['examples'], 'labels': exampleLabels}
  return labeledDict

# from internet.
# Code to find the index of a subarray.
def find_subarray(a, b):
    return [x for x in range(len(a)) if a[x:x+len(b)] == b]

In [ ]:
import pickle
import re
f = open("drive/MyDrive/Colab Notebooks/finalProj/examples.pkl","rb")
exampleDict = pickle.load(f)
f.close()
filterDict = {}
# Regularize the examples, removing non alphanumeric, lowercasing everything, and stripping whitespace.
for key, exampleSents in exampleDict.items():
  for index, sentence in enumerate(exampleSents):
    sentence = re.sub(r'[^a-zA-Z0-9-\s]', ' ', sentence)
    sentence = sentence.lower()
    sentence = sentence.strip()

    key = re.sub(r'[^a-zA-Z0-9-\s]', ' ', key)
    key = key.lower()
    key = key.strip()
    exampleSents[index] = sentence
  filterDict[key] = exampleSents

# If we do a tokenizing step, it happens here.
tokenizedDict = tokenize(filterDict)
# build out the labels
labeledDict = label(tokenizedDict)

filehandler = open("drive/MyDrive/Colab Notebooks/finalProj/labelDict.txt", 'wt')
data = str(labeledDict)
filehandler.write(data)
filehandler.close()

f = open("drive/MyDrive/Colab Notebooks/finalProj/labelDict.pkl","wb")

# write the python object (dict) to pickle file, to save it.
pickle.dump(labeledDict,f)
f.close()

In [ ]:
import re
# tokenize, configure and set up the slang negative examples.
#using the test/train datasets from assignment 2 as our negative examples.
# we are combining the test and train data into one, since we will be performing a split later in our pipeline.
negSamples = []
with open(drivePath + 'train.txt') as trainData:
  lines = trainData.readlines()
with open(drivePath + 'test.txt') as trainData:
  lines += trainData.readlines()
for line in lines:
  line = re.sub(r'[^a-zA-Z0-9-\s]', ' ', line)
  line = line.lower()
  line = line.strip()
  #TODO: if we use fancier tokenization, it will need to be implemented here too
  tokenLine = line.split()
  negSamples.append({'sent': tokenLine, 'label': ['O'] * len(tokenLine)})

In [ ]:
import pickle
# for ease of processing, convert the labeledDict (positive examples) into the same format as the negative examples, and combine them.
f = open("drive/MyDrive/Colab Notebooks/finalProj/labelDict.pkl","rb")
labelDict = pickle.load(f)
slangSamples = []
for key, slangData in labelDict.items():
  for i in range(len(slangData['examples'])):
    slangSamples.append({'sent': slangData['examples'][i], 'label': slangData['labels'][i]})

In [ ]:
import pickle
fullSamples = slangSamples + negSamples
with open(drivePath + "fullSamples.pkl","wb") as full:
  pickle.dump(fullSamples, full)
with open(drivePath + "slangSamples.pkl","wb") as slang:
  pickle.dump(slangSamples, slang)
with open(drivePath + "negSamples.pkl","wb") as neg:
  pickle.dump(negSamples, neg)
print(fullSamples[-1])

In [ ]:
import pickle
f = open("drive/MyDrive/Colab Notebooks/finalProj/examples.pkl","rb")
exampleDict = pickle.load(f)
f.close()
count = 0
for key, exampleSents in exampleDict.items():
  if "-" in key:
    count += 1
print(count)